# Install, import and GPU usage

In [5]:
import torch

num_gpus = torch.cuda.device_count()  # Get the number of available GPUs
print(f"Number of GPUs: {num_gpus}")

for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


Number of GPUs: 4
GPU 0: NVIDIA RTX A6000
GPU 1: NVIDIA RTX A6000
GPU 2: NVIDIA RTX A6000
GPU 3: NVIDIA RTX A6000


In [1]:
import pyfluidsynth
synth = pyfluidsynth.Synth()
synth.delete()
print("✅ pyfluidsynth funziona!")


ModuleNotFoundError: No module named 'pyfluidsynth'

In [20]:
"""
Binomial Diffusion for Symbolic Music Generation with MAESTRO Dataset

This script demonstrates:
1) How to set up a specific GPU device using the user's code snippet (gpu_id = 1, etc.).
2) How to load MAESTRO v3.0.0 from the folder "DASP/maestro-v3.0.0", which contains subfolders
   (2004, 2006, etc.) and the CSV file "maestro-v3.0.0.csv".
3) How to build piano-roll segments from the MAESTRO CSV, storing them in a "MaestroBinomialDiffusionDataset".
4) A UNet-like model named "MusicDiffusionGenerator".
5) Training and sampling with binomial diffusion on these segments.

"""
%pip install torch torchvision torchaudio tqdm pretty_midi pandas
%pip install pyfluidsynth

import pickle
import os
import glob
import math
import numpy as np
import pandas as pd
import pretty_midi
import IPython.display as ipd
from tqdm import tqdm
import tempfile
from scipy.io.wavfile import write
import pyfluidsynth


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# --------------------------------------------------------------------------------
# 0) Select GPU Device (gpu_id = 1, for example)
# --------------------------------------------------------------------------------
gpu_id = 1
num_gpus = torch.cuda.device_count()

if gpu_id >= num_gpus:
    raise ValueError(f"Invalid GPU ID {gpu_id}. Only {num_gpus} GPUs are available.")

os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(gpu_id)

print("Using device:", device)
print("Using GPU:", torch.cuda.get_device_name(gpu_id))
print("Device Count:", torch.cuda.device_count())
print("Current Device ID:", torch.cuda.current_device())
print("CUDA is Available:", torch.cuda.is_available())

device_props = torch.cuda.get_device_properties(gpu_id)
print("\n GPU Specifications:")
print(f"   - Name: {device_props.name}")
print(f"   - Total Memory: {device_props.total_memory / 1e9:.2f} GB")
print(f"   - Multiprocessors: {device_props.multi_processor_count}")
print(f"   - Compute Capability: {device_props.major}.{device_props.minor}")
print(f"   - Max Threads/MP: {device_props.max_threads_per_multi_processor}")
print()



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pyfluidsynth'

# Dataset

In [7]:
###############################################################################
# 1) Build Pianoroll Segments from MAESTRO
###############################################################################
def midi_to_binary_pianoroll(
    midi_path: str,
    pitch_low: int = 21,
    pitch_high: int = 108,
    resolution: int = 24,
    segment_beats: int = 16
):
    """
    Convert a single MIDI file into multiple binary piano-roll segments.
    Typically for the full 88-key range: pitch_low=21, pitch_high=108.

    Returns:
        List[np.ndarray], each shape (T, pitch_range).
        E.g. T=384 if segment_beats=16 and resolution=24 => shape=(384, 88).
    """
    print(f"\n>>> midi_to_binary_pianoroll: {midi_path}")
    print(f"    pitch_low={pitch_low}, pitch_high={pitch_high}, resolution={resolution}, segment_beats={segment_beats}")

    try:
        pm = pretty_midi.PrettyMIDI(midi_path)
        print("    Successfully loaded MIDI.")
    except Exception as e:
        print(f"    Error reading MIDI {midi_path}: {e}")
        return []

    # Get a piano roll at 'resolution' frames/sec
    piano_roll = pm.get_piano_roll(fs=resolution)  # shape=(128, num_frames)
    print(f"    Raw piano_roll shape: {piano_roll.shape} (128 x frames)")

    # Restrict pitch range
    pitch_low = max(0, pitch_low)
    pitch_high = min(127, pitch_high)
    pr_stripped = piano_roll[pitch_low:pitch_high+1, :]  # shape=(pitch_range, frames)
    print(f"    After pitch restrict => shape: {pr_stripped.shape}")

    # Binarize velocities
    pr_stripped = (pr_stripped > 0).astype(np.float32)

    # Transpose to (frames, pitch_range)
    pr_stripped = pr_stripped.T
    print(f"    Transposed => shape: {pr_stripped.shape} (frames x pitch_range)")

    # frames per segment
    frames_per_segment = segment_beats * resolution
    total_frames = pr_stripped.shape[0]
    num_segments = total_frames // frames_per_segment
    print(f"    frames_per_segment={frames_per_segment}, total_frames={total_frames}, => # segments={num_segments}")

    segments = []
    for i in range(num_segments):
        start = i * frames_per_segment
        end = start + frames_per_segment
        seg = pr_stripped[start:end, :]  # shape should be (frames_per_segment, pitch_range)
        segments.append(seg)

    return segments


def build_maestro_segments(
    maestro_csv: str,
    split: str = "train",
    pitch_low: int = 21,
    pitch_high: int = 108,
    resolution: int = 24,
    segment_beats: int = 16,
    base_dir: str = "maestro-v3.0.0"
):
    """
    Build a large list of piano-roll segments by scanning the official MAESTRO CSV,
    excluding any segment that doesn't have pitch_range=88.
    """
    print(f"\n=== build_maestro_segments ===")
    print(f"  Reading CSV: {maestro_csv}")
    print(f"  Target split: {split}")
    print(f"  pitch_low={pitch_low}, pitch_high={pitch_high}, resolution={resolution}, segment_beats={segment_beats}")
    print(f"  base_dir={base_dir}\n")

    df = pd.read_csv(maestro_csv)
    print(f"  CSV loaded, total rows = {len(df)}")

    df_split = df[df["split"] == split]
    print(f"  Rows matching split='{split}': {len(df_split)}")

    all_segments = []
    excluded_count = 0  # track how many we exclude

    for i, row in df_split.iterrows():
        midi_rel_path = row['midi_filename']  # e.g. "2004/MIDI-Unprocessed_XX_XX.mid"
        midi_abs_path = os.path.join(base_dir, midi_rel_path)

        print(f"\n  Processing row {i}, MIDI file: {midi_abs_path}")
        segs = midi_to_binary_pianoroll(
            midi_abs_path,
            pitch_low=pitch_low,
            pitch_high=pitch_high,
            resolution=resolution,
            segment_beats=segment_beats
        )
        valid_segments = []
        for seg in segs:
            # Check pitch dimension
            if seg.shape[1] == 88:
                valid_segments.append(seg)
            else:
                # Exclude the segment
                print(f"    Excluding segment with pitch dim={seg.shape[1]} (expected 88).")
                excluded_count += 1

        if valid_segments:
            print(f"  -> {len(valid_segments)} valid segments extracted (excluded some if mismatch).")
        else:
            print("  -> 0 valid segments extracted (possibly error or mismatch).")

        all_segments.extend(valid_segments)

    print(f"\n*** Finished building segments for split '{split}' ***")
    print(f"    Total segments = {len(all_segments)}")
    print(f"    Excluded segments = {excluded_count}\n")
    return all_segments



def pianoroll_to_pretty_midi(
    roll: np.ndarray,
    pitch_low: int = 21,
    fs: int = 24,
    program: int = 0
) -> pretty_midi.PrettyMIDI:
    """
    Convert a binary (frames, pitch_range) roll -> a single-instrument PrettyMIDI.
    pitch_low = the absolute MIDI pitch corresponding to roll[:,0].
    fs = frames per second for note timing.
    """
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    frames, pitch_range = roll.shape
    for p_idx in range(pitch_range):
        pitch = pitch_low + p_idx
        active = np.where(roll[:, p_idx] > 0.5)[0]
        if len(active) == 0:
            continue

        # group contiguous frames
        starts = []
        ends = []
        cur_start = active[0]
        for j in range(1, len(active)):
            if active[j] != active[j-1] + 1:
                starts.append(cur_start)
                ends.append(active[j-1])
                cur_start = active[j]
        starts.append(cur_start)
        ends.append(active[-1])

        for s, e in zip(starts, ends):
            st_t = s / fs
            end_t = (e+1) / fs
            note = pretty_midi.Note(
                velocity=100,
                pitch=pitch,
                start=st_t,
                end=end_t
            )
            instrument.notes.append(note)
    pm.instruments.append(instrument)
    return pm


def save_maestro_dataset(dataset: Dataset, out_path="data/preprocessed_maestro.pkl"):
    """
    Saves the entire Dataset object using pickle.
    """
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    print(f"=== Saving MaestroBinomialDiffusionDataset to: {out_path} ===")
    with open(out_path, "wb") as f:
        pickle.dump(dataset, f)
    print("== Save complete ==\n")


###############################################################################
# 2) MaestroBinomialDiffusionDataset
###############################################################################
class MaestroBinomialDiffusionDataset(Dataset):
    """
    Returns (x_noisy, x0, t) for each segment, for T steps of binomial noise.
    """
    def __init__(self, segments_list, num_steps=100, ratio=None):
        """
        segments_list: list of arrays shape (T, pitch_range)
        num_steps: number of diffusion steps
        ratio: prior ratio for binomial noise. If None, computed from data.
        """
        super().__init__()
        print("\n=== MaestroBinomialDiffusionDataset __init__ ===")
        print(f"  - Received {len(segments_list)} total segments.")
        print(f"  - num_steps={num_steps}, ratio={ratio}")

        self.rolls = segments_list
        self.num_steps = num_steps

        if ratio is None:
            print("  Computing average ratio of '1's across all segments...")
            total_ones = 0.0
            total_size = 0.0
            for seg in segments_list:
                total_ones += seg.sum()
                total_size += seg.size
            self.ratio = total_ones / total_size
            print(f"  => Computed ratio: {self.ratio:.6f}")
        else:
            self.ratio = ratio
            print(f"  Using provided ratio: {self.ratio}")

        print("=== Dataset initialization complete! ===\n")

    def __len__(self):
        return len(self.rolls) * self.num_steps

    def __getitem__(self, idx):
        roll_idx = idx // self.num_steps
        t = idx % self.num_steps

        x0 = self.rolls[roll_idx]  # shape (T, pitch_range)
        beta_t = (t + 1) / self.num_steps
        p_noisy = x0 * (1.0 - beta_t) + self.ratio * beta_t
        x_noisy = np.random.binomial(1, p_noisy).astype(np.float32)

        # Return shape (1, T, pitch_range) for conv2d
        return (
            torch.from_numpy(x_noisy).unsqueeze(0),
            torch.from_numpy(x0).unsqueeze(0),
            torch.tensor([t], dtype=torch.float32)
        )

In [8]:
# train_segments = build_maestro_segments(
#     maestro_csv="maestro-v3.0.0/maestro-v3.0.0.csv",
#     split="train",
#     pitch_low=21,
#     pitch_high=108,
#     resolution=24,
#     segment_beats=16,
#     base_dir="maestro-v3.0.0"
# )

# train_dataset = MaestroBinomialDiffusionDataset(train_segments, num_steps=100, ratio=0.03)

# save_maestro_dataset(train_dataset, out_path="data/preprocessed_maestro.pkl")


# Model

In [9]:
###############################################################################
# 3) MusicDiffusionGenerator (UNet-like)
###############################################################################
class MusicDiffusionGenerator(nn.Module):
    """
    A simplified UNet architecture for binomial diffusion:
    (batch, 1, T, pitch).  Pool only along time dimension.
    """
    def __init__(self, dim=48, channels=1, dim_mults=(1,2,4,4)):
        super().__init__()
        self.channels = channels

        # MLP for time embedding
        self.time_mlp = nn.Sequential(
            nn.Linear(1, 128),
            nn.ReLU(),
            nn.Linear(128, 128)
        )

        dims = [dim*m for m in dim_mults]
        in_chs = [channels] + dims[:-1]
        out_chs = dims

        self.downs = nn.ModuleList()
        # We only pool time dimension => (2,1)
        self.pool = nn.AvgPool2d(kernel_size=(2,1))

        # Down path
        for inc, outc in zip(in_chs, out_chs):
            block = nn.Sequential(
                nn.Conv2d(inc, outc, 3, padding=1),
                nn.ReLU(),
                nn.Conv2d(outc, outc, 3, padding=1),
                nn.ReLU()
            )
            self.downs.append(block)

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(dims[-1], dims[-1], 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(dims[-1], dims[-1], 3, padding=1),
            nn.ReLU()
        )

        # Ups
        self.ups = nn.ModuleList()
        reversed_in = list(reversed(dims))
        reversed_out = reversed_in[1:] + [dims[0]]

        # Only upsample time => scale_factor=(2,1)
        self.upsample = nn.UpsamplingNearest2d(scale_factor=(2,1))

        for inc, outc in zip(reversed_in, reversed_out):
            block = nn.Sequential(
                nn.Conv2d(inc*2, inc, 3, padding=1),
                nn.ReLU(),
                nn.Conv2d(inc, outc, 3, padding=1),
                nn.ReLU()
            )
            self.ups.append(block)

        self.final_conv = nn.Conv2d(dims[0], channels, kernel_size=1)

    def forward(self, x, t):
        """
        x shape: (B, 1, T, pitch_range)
        t shape: (B, 1)
        Return: same shape => (B, 1, T, pitch_range)
        """
        temb = self.time_mlp(t)  # (B, 128)
        temb = temb[..., None, None]  # (B, 128, 1, 1)

        skip_connections = []
        h = x

        # Down
        for down_block in self.downs:
            h = down_block(h)            # conv
            skip_connections.append(h)   # store skip
            h = self.pool(h)             # pool => time / 2, pitch stays same

        # Bottleneck
        h = self.bottleneck(h)

        # Up
        for up_block in self.ups:
            # upsample => time *2, pitch the same
            h = self.upsample(h)

            s = skip_connections.pop()
            # shape alignment if needed
            if h.shape[-2:] != s.shape[-2:]:
                min_h = min(h.shape[-2], s.shape[-2])
                min_w = min(h.shape[-1], s.shape[-1])
                h = h[..., :min_h, :min_w]
                s = s[..., :min_h, :min_w]

            h = torch.cat([h, s], dim=1)
            h = up_block(h)

        out = self.final_conv(h)
        return out


# Training and Sampling

In [10]:
###############################################################################
# 4) Training
###############################################################################
from tqdm import tqdm

def train_diffusion_model(
    model: nn.Module,
    dataset: Dataset,
    batch_size: int = 200,
    lr: float = 5e-5,
    epochs: int = 2,
    device: str = "cuda",
    max_steps_per_epoch: int = None  # if not None, we break after these steps
):
    """
    Train the binomial diffusion model with L1 loss, printing logs step by step.
    Uses tqdm for a progress bar, and optionally limits steps per epoch.
    """
    from torch.utils.data import DataLoader

    print(f"=== Initializing training ===")
    print(f"  - Batch size: {batch_size}")
    print(f"  - Learning rate: {lr}")
    print(f"  - Epochs: {epochs}")
    print(f"  - Device: {device}")
    print(f"  - Dataset size: {len(dataset)}")
    if max_steps_per_epoch is not None:
        print(f"  - Will stop each epoch after {max_steps_per_epoch} steps\n")

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.L1Loss()

    model.to(device)
    model.train()

    for epoch in range(epochs):
        print(f"\n>>> Starting epoch {epoch+1}/{epochs} <<<")
        # Wrap the dataloader in tqdm:
        loader = tqdm(dataloader, desc=f"Epoch {epoch+1}", total=len(dataloader))
        total_loss = 0.0
        step_count = 0

        for x_noisy, x0, t in loader:
            x_noisy = x_noisy.to(device)
            x0      = x0.to(device)
            t       = t.to(device)

            optimizer.zero_grad()
            x0_pred = model(x_noisy, t)

            loss = loss_fn(x0_pred, x0)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            step_count += 1

            # Update tqdm info
            loader.set_postfix(loss=f"{loss.item():.4f}")

            # If you want to cut short each epoch, do so here:
            if max_steps_per_epoch is not None and step_count >= max_steps_per_epoch:
                print(f"  Stopped epoch {epoch+1} after {max_steps_per_epoch} steps (early break).")
                break

        avg_loss = total_loss / step_count
        print(f"==> Epoch {epoch+1} finished. Average Loss: {avg_loss:.4f}")

    print("\n=== Training complete! ===\n")
    return model


###############################################################################
# 5) Sampling with XOR + Mask
###############################################################################
def sample_binomial_diffusion(
    model: nn.Module,
    shape=(384, 88),
    total_steps: int = 100,
    ratio: float = 0.03,
    device: str = "cuda"
):
    """
    Enhanced sampling, step by step:
      1) Start from pure binomial noise p=ratio
      2) For t in [T..1]:
         - predict x0
         - binarize x0
         - XOR with original xT
         - partial revert with mask
    Returns a final numpy array shape=(T, pitch_range).
    """
    print("=== Starting sampling process ===")
    print(f"  - shape={shape}, total_steps={total_steps}, ratio={ratio}, device={device}")

    model.eval()
    with torch.no_grad():
        # 1) create binomial noise
        xT_np = np.random.binomial(1, ratio, size=shape).astype(np.float32)
        print("  Generated initial binomial noise...")

        x_current = torch.from_numpy(xT_np).unsqueeze(0).unsqueeze(0).to(device)
        xT_tensor = x_current.clone()

        # 2) iterative
        for i in range(total_steps):
            t_val = total_steps - i - 1
            if i % 10 == 0:
                print(f"  Sampling step: {i+1}/{total_steps}, t_val={t_val}")

            t_tensor = torch.tensor([[t_val]], dtype=torch.float32, device=device)
            x0_pred = model(x_current, t_tensor)
            x0_bin = (x0_pred >= 0.5).float()

            delta = (xT_tensor != x0_bin).float()
            beta = (t_val + 1) / total_steps
            mask = torch.bernoulli(delta * beta)

            x_current = x0_bin * (1 - mask) + xT_tensor * mask

        final_roll = x_current.squeeze().detach().cpu().numpy()

    print("=== Sampling complete! Returning final roll. ===\n")
    return final_roll


###############################################################################
# 6) Save & Play
###############################################################################
def sample_pianoroll_and_save_midi(
    model: nn.Module,
    out_midi="generated_sample.mid",
    shape=(384, 88),
    total_steps=100,
    ratio=0.03,
    pitch_low=21,
    device="cuda",
    out_dir="experiments"
):
    """
    1) Sample from the diffusion model
    2) Convert to MIDI
    3) Save MIDI to disk inside out_dir (default: experiments)
    4) Return (roll, out_midi_path)
    """
    print("=== sample_pianoroll_and_save_midi ===")
    print(f"  - Output MIDI filename: {out_midi}")
    print(f"  - shape={shape}, total_steps={total_steps}, ratio={ratio}, pitch_low={pitch_low}")
    print(f"  - Saving to directory: {out_dir}\n")

    os.makedirs(out_dir, exist_ok=True)

    roll = sample_binomial_diffusion(
        model=model,
        shape=shape,
        total_steps=total_steps,
        ratio=ratio,
        device=device
    )

    print("  Converting final roll to PrettyMIDI object...")
    pm = pianoroll_to_pretty_midi(roll, pitch_low=pitch_low, fs=24)

    out_path = os.path.join(out_dir, out_midi)
    pm.write(out_path)
    print(f"  MIDI saved to: {out_path}\n")

    return roll, out_path


def generate_and_play_audio_from_model(
    model: nn.Module,
    out_name="sample_01",
    shape=(384, 88),
    pitch_low=21,
    ratio=0.3,
    total_steps=100,
    device="cuda",
    out_dir="experiments",
    fs=44100
):
    """
    Generates a sample from the model, saves the MIDI, converts to WAV,
    and returns an Audio object to play in the notebook.
    """

    print(f"🎼 Generating sample: {out_name}")
    os.makedirs(out_dir, exist_ok=True)

    # Step 1: Sample piano roll
    roll = sample_binomial_diffusion(
        model=model,
        shape=shape,
        total_steps=total_steps,
        ratio=ratio,
        device=device
    )
    print(f"  ✅ Roll generated. Shape: {roll.shape}, Sum: {roll.sum()}")

    # Step 2: Convert to PrettyMIDI
    pm = pianoroll_to_pretty_midi(roll, pitch_low=pitch_low, fs=24)

    # Step 3: Save MIDI
    midi_path = os.path.join(out_dir, f"{out_name}.mid")
    pm.write(midi_path)
    print(f"  💾 MIDI saved to: {midi_path}")

    # Step 4: Convert to WAV using fluidsynth
    try:
        audio = pm.fluidsynth(fs=fs)
        wav_path = os.path.join(out_dir, f"{out_name}.wav")
        write(wav_path, fs, audio.astype(np.int16))
        print(f"  🔊 WAV saved to: {wav_path}")

        # Step 5: Return playable audio
        return ipd.Audio(wav_path)
    
    except Exception as e:
        print(f"⚠️ Failed to synthesize audio: {e}")
        print("You may need to install `fluidsynth`. Try: pip install pyfluidsynth")
        return None


# Testing

In [11]:
def load_maestro_dataset(in_path="preprocessed_maestro.pkl"):
    """
    Loads the Dataset object from pickle.
    """
    print(f"=== Loading MaestroBinomialDiffusionDataset from: {in_path} ===")
    with open(in_path, "rb") as f:
        ds = pickle.load(f)
    print(f"== Load complete, dataset has {len(ds)} items ==\n")
    return ds


maestro_dataset = load_maestro_dataset("data/preprocessed_maestro.pkl")
print("Dataset loaded, length:", len(maestro_dataset))

from torch.utils.data import Subset

# Let's pick the first 50k samples
subset_indices = list(range(5000))

# Create a Subset
maestro_subset = Subset(maestro_dataset, subset_indices)
print("Subset length:", len(maestro_subset))


=== Loading MaestroBinomialDiffusionDataset from: data/preprocessed_maestro.pkl ===
== Load complete, dataset has 3539500 items ==

Dataset loaded, length: 3539500
Subset length: 5000


In [12]:
# Instantiate model
model = MusicDiffusionGenerator(dim=48, channels=1, dim_mults=(1,2,4,4))

In [13]:
# Train, limiting each epoch to 2k steps, so we can do a quick test:
model = train_diffusion_model(
    model,
    maestro_subset,
    batch_size=256,
    lr=1e-4,
    epochs=1,
    device=device,
    max_steps_per_epoch=2000  # remove or set None to use the full epoch
)

=== Initializing training ===
  - Batch size: 256
  - Learning rate: 0.0001
  - Epochs: 1
  - Device: cuda:1
  - Dataset size: 5000
  - Will stop each epoch after 2000 steps


>>> Starting epoch 1/1 <<<


Epoch 1: 100%|██████████| 20/20 [00:48<00:00,  2.43s/it, loss=0.0762]

==> Epoch 1 finished. Average Loss: 0.0897

=== Training complete! ===



In [16]:
# Sample + playback
audio_obj = generate_and_play_audio_from_model(
    model=model,
    out_name="my_music_sample",
    shape=(384, 88),
    pitch_low=21,
    ratio=0.3,
    total_steps=100,
    device=device,
    out_dir="experiments"
)

if audio_obj:
    display(audio_obj)

🎼 Generating sample: my_music_sample
=== Starting sampling process ===
  - shape=(384, 88), total_steps=100, ratio=0.3, device=cuda:1
  Generated initial binomial noise...
  Sampling step: 1/100, t_val=99
  Sampling step: 11/100, t_val=89
  Sampling step: 21/100, t_val=79
  Sampling step: 31/100, t_val=69
  Sampling step: 41/100, t_val=59
  Sampling step: 51/100, t_val=49
  Sampling step: 61/100, t_val=39
  Sampling step: 71/100, t_val=29
  Sampling step: 81/100, t_val=19
  Sampling step: 91/100, t_val=9
=== Sampling complete! Returning final roll. ===

  ✅ Roll generated. Shape: (384, 88), Sum: 103.0
  💾 MIDI saved to: experiments/my_music_sample.mid
⚠️ Failed to synthesize audio: fluidsynth() was called but pyfluidsynth is not installed.
You may need to install `fluidsynth`. Try: pip install pyfluidsynth


In [17]:
pm = pretty_midi.PrettyMIDI("experiments/my_music_sample.mid")
print("MIDI end time:", pm.get_end_time())
print("Number of notes:", sum(len(i.notes) for i in pm.instruments))


MIDI end time: 15.915909090909091
Number of notes: 103
